In [ ]:
!pip install kaggle-environments -U > /dev/null 2>&1
!pip install torchviz -U > /dev/null 2>&1
!cp -r ../input/lux-ai-2021/* .

In [ ]:
import numpy as np
import json
from pathlib import Path
import os
import random
from tqdm.notebook import tqdm
import pandas
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [ ]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed = 42
seed_everything(seed)

# Preprocessing

In [ ]:
# def to_label(action):
#     strs = action.split(' ')
#     unit_id = strs[1]
#     if strs[0] == 'm':
#         label = {'c': None, 'n': 0, 's': 1, 'w': 2, 'e': 3}[strs[2]]
#     elif strs[0] == 'bcity':
#         label = 4
#     else:
#         label = None
#     return unit_id, label


# def depleted_resources(obs):
#     for u in obs['updates']:
#         if u.split(' ')[0] == 'r':
#             return False
#     return True


# def create_dataset_from_json(episode_dir, team_name=['Toad Brigade']): 
#     obses = {}
#     samples = []
#     append = samples.append
#     episodes = []
#     for dir in episode_dir:
#         episodes += [path for path in Path(dir).glob('*.json') if 'output' not in path.name]
#     for filepath in tqdm(episodes): 
#         with open(filepath) as f:
#             json_load = json.load(f)

#         ep_id = json_load['info']['EpisodeId']
#         index = np.argmax([r or 0 for r in json_load['rewards']])
#         if json_load['info']['TeamNames'][index] not in team_name:
#             continue

#         for i in range(len(json_load['steps'])-1):
#             if json_load['steps'][i][index]['status'] == 'ACTIVE':
#                 actions = json_load['steps'][i+1][index]['action']
#                 obs = json_load['steps'][i][0]['observation']
                
#                 if depleted_resources(obs):
#                     break
                
#                 obs['player'] = index
#                 obs = dict([
#                     (k,v) for k,v in obs.items() 
#                     if k in ['step', 'updates', 'player', 'width', 'height']
#                 ])
#                 obs_id = f'{ep_id}_{i}'
#                 obses[obs_id] = obs
                                
#                 for action in actions:
#                     unit_id, label = to_label(action)
#                     if label is not None:
#                         append((obs_id, unit_id, label))

#     return obses, samples

In [ ]:
# load data
train = np.load('../input/lux-ai-training-data/train.npy')
val = np.load('../input/lux-ai-training-data/val.npy')
city_train = np.load('../input/lux-ai-training-data/city_train.npy',allow_pickle=True)
city_val = np.load('../input/lux-ai-training-data/city_val.npy',allow_pickle=True)
obses = np.load('../input/lux-ai-training-data/obses.npy',allow_pickle=True)

# Models

`make_input`函数用于将数据转化为输入神经网络的矩阵

`LuxDataset`是数据集管理类的封装

In [ ]:
# Input for Neural Network
def make_input(obs, unit_id, is_city = False):
    width, height = obs[3], obs[4]
    x_shift = (32 - width) // 2
    y_shift = (32 - height) // 2
    cities = {}
    #[step,updates,player,width,height]
    b = np.zeros((20, 32, 32), dtype=np.float32)
    
    for update in obs[1]:#updates
        strs = update.split(' ')
        input_identifier = strs[0]
        
        if input_identifier == 'u':
            x = int(strs[4]) + x_shift
            y = int(strs[5]) + y_shift
            wood = int(strs[7])
            coal = int(strs[8])
            uranium = int(strs[9])
            if unit_id == strs[3] and not is_city:
                # Position and Cargo
                b[:2, x, y] = (
                    1,
                    (wood + coal + uranium) / 100
                )
            else:
                # Units
                team = int(strs[2])
                cooldown = float(strs[6])
                idx = 2 + (team - obs[2]) % 2 * 3 #player
                b[idx:idx + 3, x, y] = (
                    1,
                    cooldown / 6,
                    (wood + coal + uranium) / 100
                )
        elif input_identifier == 'ct':
            # CityTiles
            team = int(strs[1])
            city_id = strs[2]
            x = int(strs[3]) + x_shift
            y = int(strs[4]) + y_shift
            idx = 8 + (team - obs[2]) % 2 * 2 #player
            b[idx:idx + 2, x, y] = (
                1,
                cities[city_id]
            )
            if is_city and unit_id == (int(strs[3]),int(strs[4])):
                b[:2, x, y] = (
                    1,
                    cities[city_id]
                )
        elif input_identifier == 'r':
            # Resources
            r_type = strs[1]
            x = int(strs[2]) + x_shift
            y = int(strs[3]) + y_shift
            amt = int(float(strs[4]))
            b[{'wood': 12, 'coal': 13, 'uranium': 14}[r_type], x, y] = amt / 800
        elif input_identifier == 'rp':
            # Research Points
            team = int(strs[1])
            rp = int(strs[2])
            b[15 + (team - obs[2]) % 2, :] = min(rp, 200) / 200  #player
        elif input_identifier == 'c':
            # Cities
            city_id = strs[2]
            fuel = float(strs[3])
            lightupkeep = float(strs[4])
            cities[city_id] = min(fuel / lightupkeep, 10) / 10
    
    # Day/Night Cycle
    b[17, :] = obs[0] % 40 / 40  #step
    # Turns
    b[18, :] = obs[0] / 360  #step
    # Map Size
    b[19, x_shift:32 - x_shift, y_shift:32 - y_shift] = 1

    return b


class LuxDataset(Dataset):
    def __init__(self, obses, samples, is_city = False):
        self.obses = obses
        self.samples = samples
        self.is_city = is_city
        
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        obs_id, unit_id, action = self.samples[idx]
        obs = self.obses[int(obs_id)]
        state = make_input(obs, unit_id, is_city = self.is_city)
        
        return state, int(action)

In [ ]:
class BasicConv2d(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, bn):
        super().__init__()
        self.conv = nn.Conv2d(
            input_dim, output_dim, 
            kernel_size=kernel_size, 
            padding=(kernel_size[0] // 2, kernel_size[1] // 2)
        )
        self.bn = nn.BatchNorm2d(output_dim) if bn else None

    def forward(self, x):
        h = self.conv(x)
        h = self.bn(h) if self.bn is not None else h
        return h

### VGGNet Model

In [ ]:
# Neural Network for Lux AI
class VGGNet(nn.Module):
    def __init__(self, output = 5):
        super().__init__()
        layers1,layers2 = 3,3
        filters1, filters2, filters3 = 128, 256, 32
        #[32*32*20]
        self.conv0 = BasicConv2d(20, filters1, (5, 5), True)
        #[32*32*128]
        self.blocks1 = nn.ModuleList([BasicConv2d(filters1, filters1, (3, 3), True) for _ in range(layers1)])
        #[32*32*128]
        self.conv1 = BasicConv2d(filters1, filters2, (5, 5), True)
        #[32*32*256]
        self.pool0 = nn.MaxPool2d(2)
        #[16*16*256]
        self.blocks2 = nn.ModuleList([BasicConv2d(filters2, filters2, (3, 3), True) for _ in range(layers2)])
        #[16*16*256]
        self.pool1 = nn.MaxPool2d(2)
        #[8*8*256]
        self.conv2 = BasicConv2d(filters2, filters3, (5, 5), True)
        #[8*8*32]
        self.pool2 = nn.MaxPool2d(2)
        #[4*4*32]
        self.flatten0 = nn.Flatten()
        #[512]
        self.fc0 = nn.Linear(4*4*32, 16, bias=False)
        #[16]
        self.fc1 = nn.Linear(16, output, bias=False)

    def forward(self, x):
        h = F.relu_(self.conv0(x))
        for block in self.blocks1:
            h = F.relu_(block(h))
        h = self.pool0(F.relu_(self.conv1(h)))
        for block in self.blocks2:
            h = F.relu_(block(h))
        h = self.conv2(self.pool1(h))
        h = self.flatten0(self.pool2(h))
        h = F.relu_(self.fc0(h))
        p = self.fc1(h)
        return p

### ResNet Model

这是基于ResNet实现的模型，网络结构基本与baseline3一致，我们对部分超参数进行了修改

In [ ]:
class ResNet(nn.Module):
    def __init__(self, output = 5):
        super().__init__()
        layers, filters = 12, 32
        self.conv0 = BasicConv2d(20, filters, (3, 3), True)
        self.blocks = nn.ModuleList([BasicConv2d(filters, filters, (3, 3), True) for _ in range(layers)])
        self.head_p = nn.Linear(filters, output, bias=False)

    def forward(self, x):
        h = F.relu_(self.conv0(x))
        for block in self.blocks:
            h = F.relu_(h + block(h))
        h_head = (h * x[:,:1]).view(h.size(0), h.size(1), -1).sum(-1)
        p = self.head_p(h_head)
        return p

In [ ]:
#选择一种模型
LuxNet = ResNet

## Train

In [ ]:
#模型训练函数
def train_model(model, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, file):
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.cuda()
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            epoch_loss = 0.0
            epoch_acc = 0
            
            dataloader = dataloaders_dict[phase]
            for item in tqdm(dataloader, leave=False):
                states = item[0].cuda().float()
                actions = item[1].cuda().long()

                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    policy = model(states)
                    loss = criterion(policy, actions)
                    _, preds = torch.max(policy, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * len(policy)
                    epoch_acc += torch.sum(preds == actions.data)

            data_size = len(dataloader.dataset)
            epoch_loss = epoch_loss / data_size
            epoch_acc = epoch_acc.double() / data_size

            print(f'Epoch {epoch + 1}/{num_epochs} | {phase:^5} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}')

        if epoch_acc > best_acc:
            traced = torch.jit.trace(model.cpu(), torch.rand(1, 20, 32, 32))
            traced.save(file)
            best_acc = epoch_acc
        scheduler.step(epoch_acc)

### Train City Model

In [ ]:
city_model = LuxNet(2)
batch_size = 128
train_loader = DataLoader(
    LuxDataset(obses, city_train, True), 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=2
)
val_loader = DataLoader(
    LuxDataset(obses, city_val, True), 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=2
)
dataloaders_dict = {"train": train_loader, "val": val_loader}
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.AdamW(city_model.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.1, patience=1, verbose=True)
train_model(city_model, dataloaders_dict, criterion, optimizer, scheduler, num_epochs = 20, file = 'city_model.pth')

### Train Worker Model

In [ ]:
model = LuxNet(5)
batch_size = 128
train_loader = DataLoader(
    LuxDataset(obses, train), 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=2
)
val_loader = DataLoader(
    LuxDataset(obses, val), 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=2
)
dataloaders_dict = {"train": train_loader, "val": val_loader}
criterion = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.1, patience=1, verbose=True)
train_model(model, dataloaders_dict, criterion, optimizer, scheduler, num_epochs = 20, file = 'model.pth')

# Submission

In [ ]:
# 从数据集复制已经训练好的模型，可选
# !cp ../input/luxaimodel/model15.pth model.pth

In [ ]:
%%writefile agent.py
import os
import numpy as np
import torch
from lux.game import Game


path = '/kaggle_simulations/agent' if os.path.exists('/kaggle_simulations') else '.'
model = torch.jit.load(f'{path}/model.pth')
city_model = torch.jit.load(f'{path}/city_model.pth')
model.eval()
city_model.eval()

def make_input(obs, unit_id, is_city = False):
    width, height = obs['width'], obs['height']
    x_shift = (32 - width) // 2
    y_shift = (32 - height) // 2
    cities = {}
    
    b = np.zeros((20, 32, 32), dtype=np.float32)
    
    for update in obs['updates']:
        strs = update.split(' ')
        input_identifier = strs[0]
        
        if input_identifier == 'u':
            x = int(strs[4]) + x_shift
            y = int(strs[5]) + y_shift
            wood = int(strs[7])
            coal = int(strs[8])
            uranium = int(strs[9])
            if unit_id == strs[3] and not is_city:
                # Position and Cargo
                b[:2, x, y] = (
                    1,
                    (wood + coal + uranium) / 100
                )
            else:
                # Units
                team = int(strs[2])
                cooldown = float(strs[6])
                idx = 2 + (team - obs['player']) % 2 * 3
                b[idx:idx + 3, x, y] = (
                    1,
                    cooldown / 6,
                    (wood + coal + uranium) / 100
                )
        elif input_identifier == 'ct':
            # CityTiles
            team = int(strs[1])
            city_id = strs[2]
            x = int(strs[3]) + x_shift
            y = int(strs[4]) + y_shift
            idx = 8 + (team - obs['player']) % 2 * 2
            b[idx:idx + 2, x, y] = (
                1,
                cities[city_id]
            )
            if is_city and unit_id == (int(strs[3]),int(strs[4])):
                b[:2, x, y] = (
                    1,
                    cities[city_id]
                )
        elif input_identifier == 'r':
            # Resources
            r_type = strs[1]
            x = int(strs[2]) + x_shift
            y = int(strs[3]) + y_shift
            amt = int(float(strs[4]))
            b[{'wood': 12, 'coal': 13, 'uranium': 14}[r_type], x, y] = amt / 800
        elif input_identifier == 'rp':
            # Research Points
            team = int(strs[1])
            rp = int(strs[2])
            b[15 + (team - obs['player']) % 2, :] = min(rp, 200) / 200
        elif input_identifier == 'c':
            # Cities
            city_id = strs[2]
            fuel = float(strs[3])
            lightupkeep = float(strs[4])
            cities[city_id] = min(fuel / lightupkeep, 10) / 10
    
    # Day/Night Cycle
    b[17, :] = obs['step'] % 40 / 40
    # Turns
    b[18, :] = obs['step'] / 360
    # Map Size
    b[19, x_shift:32 - x_shift, y_shift:32 - y_shift] = 1

    return b


game_state = None
def get_game_state(observation):
    global game_state
    
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation["player"]
    else:
        game_state._update(observation["updates"])
    return game_state


def in_city(pos):    
    try:
        city = game_state.map.get_cell_by_pos(pos).citytile
        return city is not None and city.team == game_state.id
    except:
        return False


def call_func(obj, method, args=[]):
    return getattr(obj, method)(*args)


unit_actions = [('move', 'n'), ('move', 's'), ('move', 'w'), ('move', 'e'), ('build_city',)]
def get_action(policy, unit, dest):
    for label in np.argsort(policy)[::-1]:
        act = unit_actions[label]
        pos = unit.pos.translate(act[-1], 1) or unit.pos
        if pos not in dest or in_city(pos):
            return call_func(unit, *act), pos 
            
    return unit.move('c'), unit.pos


def agent(observation, configuration):
    global game_state
    
    game_state = get_game_state(observation)    
    player = game_state.players[observation.player]
    actions = []
    
    # City Actions
    unit_count = len(player.units)
    for city in player.cities.values():
        for city_tile in city.citytiles:
            if city_tile.can_act():
                if unit_count < player.city_tile_count and not player.researched_uranium(): 
                    state = make_input(observation, (city_tile.pos.x,city_tile.pos.y), is_city = True)
                    with torch.no_grad():
                        p = city_model(torch.from_numpy(state).unsqueeze(0))
                    
                    policy = p.squeeze(0).numpy()
                    act = np.argmax(policy)
                    if act == 0:
                        actions.append(city_tile.research())
                        player.research_points += 1
                    else:#bw
                        actions.append(city_tile.build_worker())
                        unit_count += 1
                elif unit_count < player.city_tile_count:
                    actions.append(city_tile.build_worker())
                    unit_count += 1
                elif not player.researched_uranium():
                    actions.append(city_tile.research())
                    player.research_points += 1
    
    # Worker Actions
    dest = []
    for unit in player.units:
        if unit.can_act() and (game_state.turn % 40 < 30 or not in_city(unit.pos)):
            state = make_input(observation, unit.id)
            with torch.no_grad():
                p = model(torch.from_numpy(state).unsqueeze(0))

            policy = p.squeeze(0).numpy()

            action, pos = get_action(policy, unit, dest)
            actions.append(action)
            dest.append(pos)

    return actions

In [ ]:
from kaggle_environments import make

env = make("lux_ai_2021", configuration={"width": 16, "height": 16, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run(['agent.py', 'agent.py'])
env.render(mode="ipython", width=1200, height=800)

In [ ]:
!tar -czf submission.tar.gz *